In [1]:
import argparse
import os, sys
import time
import datetime

# Import pytorch dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms # useful library for preprocessing
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm

# Import matplotlib dependencies
import matplotlib
import matplotlib.pyplot as plt

import numpy as np

# You cannot change this line.
from tools.dataloader import CIFAR10

In [2]:
""" 
Assignment 2(a)
Build the LeNet-5 model by following table 1 or figure 1.

You can also insert batch normalization and leave the LeNet-5 
with batch normalization here for assignment 3(c).
""" 
# Create the neural network module: LeNet-5
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0), # in_channels, out_channels, kernel_size, stride=1, padding=0
            nn.BatchNorm2d(6), # insert batch normalization BEFORE ReLU
            nn.ReLU()
        )
        self.layer2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer3 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )
        self.layer4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer5 = nn.Sequential(
            nn.Linear(5 * 5 * 16, 120),
            nn.BatchNorm1d(120),
            nn.ReLU()
        )
        self.layer6 = nn.Sequential(
            nn.Linear(120, 84),
            nn.BatchNorm1d(84),
            nn.ReLU()
        )
        self.layer7 = nn.Linear(84, 10)
        # CrossEntropyLoss() loss function already implements nn.Softmax(), so no need to include here
        
    def forward(self, x):
        out1 = self.layer1(x)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)
        out4 = self.layer4(out3)
        out4 = out4.view(out4.size(0),-1)
        out5 = self.layer5(out4)
        out6 = self.layer6(out5)
        y_pred = self.layer7(out6)
        return y_pred


In [3]:
"""
Hyperparameter optimization in assignment 4(a), 4(b) can be 
conducted here.
Be sure to leave only your best hyperparameter combination
here and comment the original hyperparameter settings.
"""

"""
# Initial hyperparameter values
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 100
INITIAL_LR = 0.01
MOMENTUM = 0.9
REG = 1e-4
EPOCHS = 30"""

# Setting some hyperparameters
TRAIN_BATCH_SIZE = 64
VAL_BATCH_SIZE = 100
INITIAL_LR = 0.035
MOMENTUM = 0.9
REG = 3e-4
EPOCHS = 100
DATAROOT = "./data"
CHECKPOINT_PATH = "./saved_model"

**Your answer:**

In [4]:
"""
Assignment 2(b)
Write functions to load dataset and preprocess the incoming data. 
We recommend that the preprocess scheme \textbf{must} include 
normalize, standardization, batch shuffling to make sure the training 
process goes smoothly. 
The preprocess scheme may also contain some data augmentation methods 
(e.g., random crop, random flip, etc.). 

Reference value for mean/std:

mean(RGB-format): (0.4914, 0.4822, 0.4465)
std(RGB-format): (0.2023, 0.1994, 0.2010)


NOTE: Considering this process has strong corrlelation with assignment 3(b), 
please leave the data preprocessing method which can achieve the highest 
validation accuracy here. You can include your original data augmentation
method as comments and denotes the accuracy difference between thest two 
methods.
"""
# Specify preprocessing function.
# Reference mean/std value for 

# Data augmentation for training
# ToTensor() HAS to be placed after all PIL transforms
transform_train = transforms.Compose([ # Compose multiple transformations together
    transforms.RandomHorizontalFlip(), # probability=0.5
    transforms.ToTensor(), # convert np array to torch tensor; also normalizes array to [0,1]
    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010)) # standardize array; mean , std (for each channel)
    ])
    # could also try transforms.RandomErasing()
    
# Data augmentation for inference mode
transform_val = transforms.Compose([ 
    transforms.ToTensor(), 
    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))
    ])

**Your answer:**

In [5]:
# Call the dataset Loader
trainset = CIFAR10(root=DATAROOT, train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=1) # applies batch shuffle here
valset = CIFAR10(root=DATAROOT, train=False, download=True, transform=transform_val)
valloader = torch.utils.data.DataLoader(valset, batch_size=VAL_BATCH_SIZE, shuffle=False, num_workers=1)

Using downloaded and verified file: ./data/cifar10_trainval.tar.gz
Extracting ./data/cifar10_trainval.tar.gz to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data/cifar10_trainval.tar.gz
Extracting ./data/cifar10_trainval.tar.gz to ./data
Files already downloaded and verified


In [6]:
# Specify the device for computation
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = LeNet5()
net = net.to(device)
if device =='cuda':
    print("Train on GPU...")
else:
    print("Train on CPU...")

Train on GPU...


In [7]:
# FLAG for loading the pretrained model
TRAIN_FROM_SCRATCH = False
# Code for loading checkpoint and recover epoch id.
CKPT_PATH = "./saved_model/model.h5"
def get_checkpoint(ckpt_path):
    try:
        ckpt = torch.load(ckpt_path)
    except Exception as e:
        print(e)
        return None
    return ckpt

ckpt = get_checkpoint(CKPT_PATH)
if ckpt is None or TRAIN_FROM_SCRATCH:
    if not TRAIN_FROM_SCRATCH:
        print("Checkpoint not found.")
    print("Training from scratch ...")
    start_epoch = 0
    current_learning_rate = INITIAL_LR
else:
    print("Successfully loaded checkpoint: %s" %CKPT_PATH)
    net.load_state_dict(ckpt['net'])
    start_epoch = ckpt['epoch'] + 1
    current_learning_rate = ckpt['lr']
    print("Starting from epoch %d " %start_epoch)

print("Starting from learning rate %f:" %current_learning_rate)

Successfully loaded checkpoint: ./saved_model/model.h5
Starting from epoch 71 
Starting from learning rate 0.017258:


In [8]:
"""
Assignment 2(c)
In the targeted classification task, we use cross entropy loss with L2 
regularization as the learning object.
You need to formulate the cross-entropy loss function in PyTorch.
You should also specify a PyTorch Optimizer to optimize this loss function.
We recommend you to use the SGD-momentum with an initial learning rate 0.01 
and momentum 0.9 as a start.
"""
# Create loss function and specify regularization
criterion = nn.CrossEntropyLoss()
# Add optimizer
optimizer = optim.SGD(net.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)

In [9]:
"""
Assignment 3(a)
Start the training process over the whole CIFAR-10 training dataset. 
For sanity check, you are required to report the initial loss value at 
the beginning of the training process and briefly justify this value. 
Run the training process for \textbf{a maximum of 30} epochs and you 
should be able to reach around \textbf{65\%} accuracy on the validation 
dataset.
"""
# Start the training/validation process
# The process should take about 5 minutes on a GTX 1070-Ti
# if the code is written efficiently.
global_step = 0
best_val_acc = 0
initialPass = True

train_accs = []
val_accs = []

for i in range(start_epoch, EPOCHS):
    print(datetime.datetime.now())
    # Switch to train mode
    net.train()
    print("Epoch %d:" %i)

    total_examples = 0
    correct_examples = 0

    train_loss = 0
    train_acc = 0
    # Train the training dataset for 1 epoch.
    print(len(trainloader))
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        # Copy inputs to device [needs to be consistent with the device that model is loaded in]
        inputs = inputs.to(device=device, dtype=torch.float)
        targets = targets.to(device=device, dtype=torch.long)
        # Zero the gradient
        optimizer.zero_grad()
        # Generate output
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        # Now backward loss
        loss.backward()
        # Apply gradient
        optimizer.step()
        # Calculate predicted labels
        # Outputs of neurons for each input listed across dimension 1
        # Dimension 0 lists neuron outputs for all inputs
        # Neuron with largest output value is understood as y_pred
        # Max outputs (value, index) -- in this case, the index (ie neuron ID) is what we're after
        _, predicted = outputs.max(1)
        # Calculate accuracy
        total_examples += TRAIN_BATCH_SIZE
        correct_examples += (targets == predicted).sum() # .sum() all values that match between targets and predicted

        if initialPass == True:
            print("Initial loss = " + str(loss))
            initialPass = False
        train_loss += loss

        global_step += 1
        if global_step % 100 == 0:
            avg_loss = train_loss.float() / (batch_idx + 1)
        pass
    avg_acc = correct_examples.float() / total_examples # need .float() since both are ints by default
    train_accs.append(avg_acc)
    print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))
    print(datetime.datetime.now())
    # Validate on the validation dataset
    print("Validation...")
    total_examples = 0
    correct_examples = 0
    
    ## Validation process ##
    net.eval()

    val_loss = 0
    val_acc = 0
    # Disable gradient during validation
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(valloader):
            # Copy inputs to device
            inputs = inputs.to(device=device, dtype=torch.float)
            targets = targets.to(device=device, dtype=torch.long)
            # Zero the gradient
            optimizer.zero_grad()
            # Generate output from the DNN.
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            # Calculate predicted labels
            _, predicted = outputs.max(1)
            # Calculate accuracy
            total_examples += VAL_BATCH_SIZE
            correct_examples += (targets == predicted).sum()
            val_loss += loss

    avg_loss = val_loss.float() / len(valloader)
    avg_acc = correct_examples.float() / total_examples
    val_accs.append(avg_acc)
    print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))

    """
    Assignment 4(b)
    Learning rate is an important hyperparameter to tune. Specify a 
    learning rate decay policy and apply it in your training process. 
    Briefly describe its impact on the learning curveduring your 
    training process.    
    Reference learning rate schedule: 
    decay 0.98 for every 2 epochs. You may tune this parameter but 
    minimal gain will be achieved.
    Assignment 4(c)
    As we can see from above, hyperparameter optimization is critical 
    to obtain a good performance of DNN models. Try to fine-tune the 
    model to over 70% accuracy. You may also increase the number of 
    epochs to up to 100 during the process. Briefly describe what you 
    have tried to improve the performance of the LeNet-5 model.
    """
    DECAY_EPOCHS = 2
    DECAY = 0.98
    if i % DECAY_EPOCHS == 0 and i != 0:
        current_learning_rate *= DECAY
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
            
        print("Current learning rate has decayed to %f" %current_learning_rate)
    
    # Save for checkpoint
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
        if not os.path.exists(CHECKPOINT_PATH):
            os.makedirs(CHECKPOINT_PATH)
        print("Saving ...")
        state = {'net': net.state_dict(),
                 'epoch': i,
                 'lr': current_learning_rate}
        torch.save(state, os.path.join(CHECKPOINT_PATH, 'model.h5'))

print("Optimization finished.")

2019-09-30 23:48:06.289449
Epoch 71:
704
Initial loss = tensor(0.6033, device='cuda:0', grad_fn=<NllLossBackward>)
Training loss: 0.6906, Training accuracy: 0.7541
2019-09-30 23:48:23.148930
Validation...
Validation loss: 0.9373, Validation accuracy: 0.6828
Saving ...
2019-09-30 23:48:24.264146
Epoch 72:
704
Training loss: 0.6868, Training accuracy: 0.7547
2019-09-30 23:48:34.122280
Validation...
Validation loss: 0.8745, Validation accuracy: 0.6986
Current learning rate has decayed to 0.016912
Saving ...
2019-09-30 23:48:35.190012
Epoch 73:
704


KeyboardInterrupt: 

Max Validation Accuracy: tensor(0.7238, device='cuda:0')
Final Training Accuracy: tensor(0.8218, device='cuda:0')


In [11]:
# Save results
with open('report_data/train_acc_curr.txt', 'w') as filehandle:
    for val in train_accs:
        filehandle.write('%s\n' % val)
with open('report_data/val_acc_curr.txt', 'w') as filehandle:
    for val in val_accs:
        filehandle.write('%s\n' % val)

# plot results 
fig, ax = plt.subplots()
ax.plot(range(len(train_accs)), train_accs)
ax.plot(range(len(val_accs)), val_accs)
ax.set_xlabel('Epoch', fontsize = 'large')
ax.set_ylabel('Accuracy', fontsize = 'large')
ax.set_title('LeNet-5')
ax.legend(['Train Accuracy','Val Accuracy'], loc=4)
ax.grid(True)
#plt.xlim(80,100)
#plt.ylim(0.65,0.82)
plt.show()

print("Max Validation Accuracy: "+str(max(val_accs)))
print("Final Training Accuracy: "+str(train_accs[-1]))